In [6]:
# Install necessary libraries (run this in your terminal if not already installed)
# pip install librosa scikit-learn numpy

# Import libraries
import librosa
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
import os

# Define the full note mapping with microtones (adjust frequencies as needed)
note_mapping = {
    'sa': (240, 260),
    're': (260, 290),
    're#': (290, 310),  # Microtone
    'ga': (310, 330),
    'ma': (330, 370),
    'ma#': (370, 390),  # Microtone
    'pa': (390, 420),
    'dha': (420, 470),
    'ni': (470, 520),
    'ni#': (520, 540)   # Microtone
}

# Generate dataset using random.uniform
def generate_dataset(note_mapping, samples_per_note=200):
    frequencies = []
    labels = []
    for note, (low, high) in note_mapping.items():
        # Generate random frequencies for the note
        random_freqs = np.random.uniform(low, high, samples_per_note)
        frequencies.extend(random_freqs)
        labels.extend([note] * samples_per_note)  # Add corresponding labels
    return np.array(frequencies), np.array(labels)

# Map frequency to note
def frequency_to_note(freq):
    for note, (low, high) in note_mapping.items():
        if low <= freq <= high:
            return note
    return None

# Generate features for training
def extract_features(audio_path, sr=22050, hop_length=512):
    try:
        y, _ = librosa.load(audio_path, sr=sr)
        stft = librosa.stft(y, n_fft=2048, hop_length=hop_length)
        pitches, magnitudes = librosa.piptrack(S=np.abs(stft), sr=sr)
        features = []

        for i in range(pitches.shape[1]):
            pitch_slice = pitches[:, i]
            mag_slice = magnitudes[:, i]
            if mag_slice.any():
                freq = pitch_slice[np.argmax(mag_slice)]
                if freq > 0:
                    features.append(freq)
        return features
    except Exception as e:
        print(f"Error extracting features: {e}")
        return []

# Train a machine learning model
def train_model():
    # Generate a dataset using the new generator
    frequencies, labels = generate_dataset(note_mapping)

    # Reshape frequencies for training
    X = frequencies.reshape(-1, 1)
    y = labels

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Random Forest Classifier
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    print("\nClassification Report:\n", classification_report(y_test, predictions))

    # Save the model
    with open("note_classifier.pkl", "wb") as f:
        pickle.dump(model, f)

    return model

# Predict notes in an audio file
def detect_notes_with_model(audio_path, model_path="note_classifier.pkl", sr=22050, hop_length=512):
    try:
        # Load the pre-trained model
        if not os.path.exists(model_path):
            print(f"Model file '{model_path}' not found. Train the model first.")
            return

        with open(model_path, "rb") as f:
            model = pickle.load(f)

        # Extract features from the audio file
        features = extract_features(audio_path, sr=sr, hop_length=hop_length)
        if not features:
            print("No features extracted. Please check the audio file.")
            return

        # Predict notes for each frequency
        predicted_notes = model.predict(np.array(features).reshape(-1, 1))

        # Time per frame (hop_length / sample_rate)
        time_per_frame = hop_length / sr

        # Collect detected notes with durations
        detected_notes = []
        previous_note = None
        note_start_time = 0

        for i, note in enumerate(predicted_notes):
            current_time = i * time_per_frame
            if note != previous_note:
                if previous_note is not None:
                    duration = current_time - note_start_time
                    detected_notes.append({
                        "note": previous_note,
                        "start_time": round(note_start_time, 2),
                        "duration": round(duration, 2)
                    })
                previous_note = note
                note_start_time = current_time

        # Add the last note
        if previous_note is not None:
            duration = len(predicted_notes) * time_per_frame - note_start_time
            detected_notes.append({
                "note": previous_note,
                "start_time": round(note_start_time, 2),
                "duration": round(duration, 2)
            })

        # Print detected notes
        print("\nDetected Notes with Durations:")
        for note in detected_notes:
            print(f"Note: {note['note']}, Start Time: {note['start_time']} seconds, Duration: {note['duration']} seconds")

    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    # Train the model (run this once)
    train_model()

    # Provide the path to the audio file
    audio_path = input("Enter the path to the audio file: ").strip()
    if os.path.exists(audio_path):
        detect_notes_with_model(audio_path)
    else:
        print(f"File '{audio_path}' does not exist. Please check the path.")



Classification Report:
               precision    recall  f1-score   support

         dha       1.00      1.00      1.00        44
          ga       1.00      1.00      1.00        36
          ma       1.00      1.00      1.00        34
         ma#       1.00      1.00      1.00        34
          ni       1.00      1.00      1.00        48
         ni#       1.00      1.00      1.00        42
          pa       1.00      1.00      1.00        33
          re       1.00      1.00      1.00        47
         re#       1.00      1.00      1.00        46
          sa       1.00      1.00      1.00        36

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400


Detected Notes with Durations:
Note: re, Start Time: 0.0 seconds, Duration: 0.09 seconds
Note: ma, Start Time: 0.09 seconds, Duration: 0.02 seconds
Note: ga, Start Time: 0.12 seconds, Duration: 0.02 seconds
Note: ma